In [1]:
from nustar_gen.radial_profile import find_source, make_radial_profile, optimize_radius_snr
from nustar_gen.wrappers import make_image

from astropy.wcs import WCS
from astropy.io import fits
from astropy.coordinates import SkyCoord
import numpy as np

In [2]:
from nustar_gen import info
obs = info.Observation(path='../data/', seqid='30001039002')
obs.exposure_report()
obs.science_files

Exposure for FPMA, mode 01 is:      39.98 ks
Exposure for FPMA, mode 03 is:   0.004635 ks
Exposure for FPMA, mode 04 is:        0.0 ks
Exposure for FPMA, mode 06 is:      4.201 ks

Exposure for FPMB, mode 01 is:      39.97 ks
Exposure for FPMB, mode 03 is:   0.004647 ks
Exposure for FPMB, mode 04 is:        0.0 ks
Exposure for FPMB, mode 06 is:      4.204 ks



{'A': ['../data/30001039002/event_cl/nu30001039002A01_cl.evt',
  '../data/30001039002/event_cl/nu30001039002A06_chu13_N_cl.evt',
  '../data/30001039002/event_cl/nu30001039002A06_chu3_N_cl.evt',
  '../data/30001039002/event_cl/nu30001039002A06_cl.evt'],
 'B': ['../data/30001039002/event_cl/nu30001039002B01_cl.evt',
  '../data/30001039002/event_cl/nu30001039002B06_chu13_N_cl.evt',
  '../data/30001039002/event_cl/nu30001039002B06_chu3_N_cl.evt',
  '../data/30001039002/event_cl/nu30001039002B06_cl.evt']}

In [ ]:
mod='A'
infile = obs.science_files[mod][0]
print(infile)
full_range = make_image(infile, elow = 3, ehigh = 80, clobber=True)
coordinates = find_source(full_range, show_image = True, filt_range=3)

In [ ]:
# Get the WCS header and convert the pixel coordinates into an RA/Dec object
hdu = fits.open(full_range, uint=True)[0]
wcs = WCS(hdu.header)

# The "flip" is necessary to go to [X, Y] ordering from native [Y, X] ordering, which wcs seems to require
world = wcs.all_pix2world(np.flip(coordinates), 0)
ra = world[0][0]
dec = world[0][1]
target = SkyCoord(ra, dec, unit='deg', frame='fk5')
print(target)
obj_j2000 = SkyCoord(hdu.header['RA_OBJ'], hdu.header['DEC_OBJ'], unit = 'deg', frame ='fk5')

# How far are we from the J2000 coordinates? If <15 arcsec, all is okay
sep = target.separation(obj_j2000)
print(sep)


In [ ]:
# Now the radial image parts.

# Make the radial image for the full energy range (or whatever is the best SNR)
full_range = make_image(infile, elow = 3, ehigh = 80, clobber=True)
rind, rad_profile, radial_err, psf_profile = make_radial_profile(full_range, show_image=False,
                                                                 coordinates = coordinates)

In [ ]:
# Pick energy ranges that you want to check.

# Note that this formalism breaks down when the source isn't detected, so use your best judgement here.

# Below should be used as a "best guess" when choosing a radius for spectral extraction.

# For the 3-20 keV case, the source dominates out the edge of the FoV (and the assumptoons about the PSF
# start to break down in the fit).

# This a soft source (LMC X-1), so for 20-30 keV we already see that we need to restrict the radius that we
# use so that we're not just adding noise to the spectrum.

pairs = [[3, 20], [20, 30], [30, 40], [40, 50], [50, 80]]
coordinates = find_source(full_range, show_image = False)
for pair in pairs:
    test_file = make_image(infile, elow = pair[0], ehigh = pair[1], clobber=True)
    rind, rad_profile, radial_err, psf_profile = make_radial_profile(test_file, show_image=False,
                                                                     coordinates = coordinates)
    rlimit = optimize_radius_snr(rind, rad_profile, radial_err, psf_profile, show=True)
    print('Radius of peak SNR for {} to {} keV: {}'.format(
            pair[0], pair[1], rlimit))

In [ ]:
import regions
import astropy.units as u

In [ ]:
source_reg = [regions.CircleSkyRegion(center=target, radius=60*u.arcsec)]
outfile = '../data/30001039002/event_cl/srcB01.reg'
regions.write_ds9(source_reg, outfile, radunit='arcsec')